# The Python Debugger

27.3 pdb — The Python Debugger
https://docs.python.org/3/library/pdb.html

pdb – Interactive Debugger
https://pymotw.com/2/pdb/index.html

Python 代码调试技巧
https://www.ibm.com/developerworks/cn/linux/l-cn-pythondebugger/


The module pdb defines an interactive source code debugger for Python programs. It supports setting (conditional) breakpoints and single stepping at the source line level, inspection of stack frames, source code listing, and evaluation of arbitrary Python code in the context of any stack frame. It also supports post-mortem debugging and can be called under program control.

The debugger is extensible – it is actually defined as the class Pdb. This is currently undocumented but easily understood by reading the source. The extension interface uses the modules bdb and cmd.

The debugger’s prompt is (Pdb). 

Typical usage to run a program under control of the debugger is:




In [ ]:
import pdb
import mymodule
pdb.run('mymodule.test()')


pdb implements an interactive debugging environment for Python programs. It includes features to let you pause your program, look at the values of variables, and watch program execution step-by-step, so you can understand what your program actually does and find bugs in the logic.
<ol>
<li>Starting the Debugger
    <ul>
    <li>From the Command Line
    <li>Within the Interpreter
    <li>From Within Your Program
    <li>After a Failure
    </ul>
    
<li>Controlling the Debugger
    <ul>
    <li>Navigating the Execution Stack
    <li>Examining Variables on the Stack
    <li>Stepping Through Your Program
    </ul>
<li>Breakpoints
<li>Changing Execution Flow
<li>Customizing the Debugger with Aliases
<li>Saving Configuration Settings
</ol>

## 1.1 Starting the Debugger

The first step to using `pdb` is causing the interpreter to enter the debugger when you want it to.

There are a few different ways to do that, depending on your starting conditions and what you need to debug.
<ol>
<li>From the Command Line
<li>Within the Interpreter
<li>From Within Your Program
<li>After a Failure
</ol>


### 1 From the Command Line

The most straightforward way to use the debugger is to run it from the command line, giving it your own program as input so it knows what to run.


In [ ]:
# ../code/chapter27.3/pdb_script.py
class MyObj(object):

    def __init__(self, num_loops):
        self.count = num_loops

    def go(self):
        for i in range(self.count):
            print(i)
        return

if __name__ == '__main__':
    MyObj(5).go()

Running the debugger from the command line causes it to load your source file and stop execution on the first statement it finds. In this case, it stops before evaluating the definition of the class MyObj on line 7.

In [ ]:
%pdb ../code/chapter27.3.pdb_script.py

### 2 Within the Interpreter

Many Python developers work with the interactive interpreter while developing early versions of modules because it lets them experiment more iteratively without the save/run/repeat cycle needed when creating standalone scripts. To run the debugger from within an interactive interpreter, use run() or runeval().

```
$ python
Python 2.7 (r27:82508, Jul  3 2010, 21:12:11)
[GCC 4.0.1 (Apple Inc. build 5493)] on darwin
Type "help", "copyright", "credits" or "license" for more information.
>>> import pdb_script
>>> import pdb
>>> pdb.run('pdb_script.MyObj(5).go()')
> <string>(1)<module>()
(Pdb)
```

Within the Interpreter:Ipython


import sys;
sys.path.append("../code/chapter27.3/")
from pdb_script import *

import pdb
pdb.run('pdb_script.MyObj(5).go()')
MyObj(5).go()

The argument to `run()` is <b>a string expression</b> that can be evaluated by the Python interpreter.

The debugger will parse it, then pause execution just before the first expression evaluates.

You can use the debugger commands described below to navigate and control the execution.

### 3 From Within Your Program¶

Both of the previous examples assume you want to start the debugger <b>at the beginning</b> of your program. 

For a long-running process where the problem <b>appears much later</b> in the program execution, 

it will be more convenient to start the debugger from inside your program using <b>set_trace()</b>.


In [ ]:
import pdb

class MyObj(object):

    def __init__(self, num_loops):
        self.count = num_loops

    def go(self):
        for i in range(self.count):
            # start the debugger from inside your program using set_trace().
            pdb.set_trace()
            print(i)
        return

if __name__ == '__main__':
    MyObj(5).go()


Line `pdb.set_trace()`of the sample script triggers the debugger at that point in execution.

In [ ]:
%run ..\code\chapter27.3\pdb_set_trace.py

`set_trace()` is just a Python function, so you can call it <b>at any point</b> in your program.

This lets you enter the debugger based on conditions inside your program, including from an exception handler or via a specific branch of a control statement.

### 4 After a Failure

Debugging a failure after a program terminates is called <b>post-mortem debugging</b>. 

`pdb` supports post-mortem debugging through the `pm()` and `post_mortem()` functions.


In [ ]:
# pdb_post_mortem.py
class MyObj(object):

    def __init__(self, num_loops):
        self.count = num_loops

    def go(self):
        for i in range(self.num_loops):
            print(i)
        return

Here the incorrect attribute name `self.num_loops` triggers an `AttributeError` exception, causing execution to stop. 


In [ ]:
MyObj(5).go()

In [ ]:
```
>>> from pdb_post_mortem import MyObj
>>> MyObj(5).go()
Traceback (most recent call last):
  File "<pyshell#1>", line 1, in <module>
    MyObj(5).go()
  File "D:\SEUCourse\SE_ThermalEnergy\git\Py03013052\code\Chapter27.3\pdb_post_mortem.py", line 10, in go
    for i in range(self.num_loops):
AttributeError: 'MyObj' object has no attribute 'num_loops'
>>> 
```

`pm()` looks for the active traceback and starts the debugger at the point in the call stack where the exception occurred

```
>>> import pdb
>>> pdb.pm()
> d:\seucourse\se_thermalenergy\git\py03013052\code\chapter27.3\pdb_post_mortem.py(10)go()
-> for i in range(self.num_loops):
(Pdb)
```

In [ ]:
>\py03013052\code\chapter27.3\pdb_set_trace.py(13)go() 
pdb_set_trace.py(13)go()
-> print(i)

(Pdb) where

<string>(1)<module>()
  c:\python35\lib\idlelib\run.py(124)main()
-> ret = method(*args, **kwargs)
  c:\python35\lib\idlelib\run.py(351)runcode()
-> exec(code, self.locals)
  d:\seucourse\se_thermalenergy\git\py03013052\code\chapter27.3\pdb_set_trace.py(17)<module>()
-> MyObj(5).go()
> d:\seucourse\se_thermalenergy\git\py03013052\code\chapter27.3\pdb_set_trace.py(13)go()
-> print(i)
(Pdb) 

## 1.2  Controlling the Debugger

You interact with the debugger using a small command language that lets you move around the call stack, examine and change the values of variables, and control how the debugger executes your program. 

The interactive debugger uses `readline` to accept commands.

Entering a blank line `re-runs` the previous command again, unless it was a list operation.

 <ul>
    <li>Navigating the Execution Stack
    <li>Examining Variables on the Stack
    <li>Stepping Through Your Program
 </ul>

### 1.2.1 Navigating the Execution Stack

At any point while the debugger is running you can use <b>where (abbreviated w)</b> to find out exactly what line is being executed and where on the call stack you are. 

<b>context around the current location</b>

  List source code for the current file. Without arguments, list 11 lines around the current line or continue the previous listing. With . as argument, list 11 lines around the current line. With one argument, list 11 lines around at that line. With two arguments, list the given range; if the second argument is less than the first, it is interpreted as a count.
```
l(ist) [first[, last]]
```    
```
（pdb) List 
（pdb) List 8
（pdb) List 3,8
```
 List all source code for the current function or frame. Interesting lines are marked as for list. 
```
ll（longlist）
```
```
（pdb) ll
（pdb) longlist
```


<b>Move between frames within the current call stack using up  and down </b>. 
```
（pdb) up
（pdb) down
```
<hr style="height:1px;color:blue"/>

In this case, the module `pdb_set_trace.py` line 13 in the `go()` method.


To add more <b>context</b> around the current location, use <b>list</b>.

In [ ]:
```
(Pdb) list

  8  	        self.count = num_loops
  9  	
 10  	    def go(self):
 11  	        for i in range(self.count):
 12  	            pdb.set_trace()
 13  ->	            print(i)   # the current line
 14  	        return
 15  	
 16  	if __name__ == '__main__':
 17  	    MyObj(5).go()
[EOF]
(Pdb) 
```

The default is to list <b>11 lines</b> around the current line (<b>five before and five after</b>). 

Using `list` with <b>a single numerical argument</b> lists 11 lines around <b>that line(a single numerical argument) </b> instead of the current line.



In [ ]:
#  lists 11 lines around that line 8
((Pdb) list 8
  3  	import pdb
  4  	
  5  	class MyObj(object):
  6  	
  7  	    def __init__(self, num_loops):
  8  	        self.count = num_loops
  9  	
 10  	    def go(self):
 11  	        for i in range(self.count):
 12  	            pdb.set_trace()
 13  ->	            print(i)
(Pdb) 

If <b>list</b> receives two arguments, it interprets them as <b>the first and last lines</b> to include in its output.

In [ ]:
Pdb) list 3,8
  3  	import pdb
  4  	
  5  	class MyObj(object):
  6  	
  7  	    def __init__(self, num_loops):
  8  	        self.count = num_loops
(Pdb) 

ll | longlist

List all source code for the current function or frame. Interesting lines are marked as for list.


In [ ]:
-> print(i)
(Pdb) ll
 10  	    def go(self):
 11  	        for i in range(self.count):
 12  	            pdb.set_trace()
 13  ->	            print(i)
 14  	        return

Move between frames within the current call stack using <b>up </b> and  <b>down </b>. 

up (abbreviated u) moves towards older frames on the stack. 

down (abbreviated d) moves towards newer frames.

In [ ]:
(Pdb) up
> <string>(1)<module>()
(Pdb) down
> c:\python35\lib\idlelib\run.py(124)main()
-> ret = method(*args, **kwargs)
(Pdb) 

### 1.2.2 Examining Variables on the Stack

Each frame on the stack maintains a set of variables, including values local to the function being executed and global state information. 

<b>pdb</b> provides several ways to examine the contents of those variables.
<ol>
<li><b>a(rgs)</b>: Print the argument list of the current function.
<li><b>p</b> `expression` : Evaluate the expression in the current context and print its value
<li><b>pp</b> `expression`: the value of the expression is “pretty printed”  using the `pprint` module. for more complicated values
<li><b>!</b>  `statement` : Execute the (one-line) statement in the context of the current stack frame. 
</ol>

The `args' command (abbreviated a) prints all of the arguments to the function active in the current frame. 

This example also uses a recursive function to show what a deeper stack looks like when printed by where.

In [ ]:

> d:\seucourse\se_thermalenergy\git\py03013052\code\chapter27.3\pdb_function_arguments.py(10)recursive_function()
-> print(output)
(Pdb) args
n = 0
output = 'to be printed'
(Pdb) 

The <b>p</b> command evaluates an expression given as argument and prints the result.

You can also use Python’s `print` statement, but that is passed through to the interpreter to be executed rather than running as a command in the debugger.

In [ ]:
(Pdb) p n
0
(Pdb) print(n)
0
(Pdb) 

In [ ]:
import pdb

with open('lorem.txt', 'rt') as f:
    lines = f.readlines()

pdb.set_trace()

Printing the variable lines with `p` results in output that is difficult to read because it wraps awkwardly. 

`pp` uses `pprint` to format the value for clean printing.

In [ ]:
(Pdb) p lines
['Lorem ipsum dolor sit amet, consectetuer adipiscing elit. \n', 'Donec egestas, enim et consectetuer ullamcorper, lectus \n', 'ligula rutrum leo, a elementum elit tortor eu quam.\n']
(Pdb) pp lines
['Lorem ipsum dolor sit amet, consectetuer adipiscing elit. \n',
 'Donec egestas, enim et consectetuer ullamcorper, lectus \n',
 'ligula rutrum leo, a elementum elit tortor eu quam.\n']
(Pdb) 

Similarly, <b>prefixing an expression with !</b> passes it to the Python interpreter to be evaluated. 

You can use this feature to <b>execute arbitrary Python statements</b>, including modifying variables. 

This example changes the value of output before letting the debugger continue running the program. 

The next statement after the call to `set_trace()` prints the value of output, showing the modified value.

For more complicated values such as nested or large data structures, use `pp` to “pretty print” them. 

This program reads several lines of text from a file.

In [ ]:
(Pdb) print(output) 
to be printed
(Pdb) !output
'to be printed'
(Pdb) !output='Changed value'
(Pdb) c
Changed value
>>> 

Use <b>step</b> to execute the current line and then stop at the next execution point – either the first statement inside a function being called or the next line of the current function


In [ ]:
$ python pdb_step.py
> /Users/dhellmann/Documents/PyMOTW/src.pdb/PyMOTW/pdb/pdb_step.py(17)<module>()
-> f(5)

The interpreter pauses at the call to `set_trace()` and gives control to the debugger. 

The first step causes the execution to enter `f()`.

In [ ]:
(Pdb) step
--Call--
> .../pdb_step.py(9)f()
-> def f(n):

One more step moves execution to the first line of f() and starts the loop.

In [ ]:
(Pdb) step
> /Users/dhellmann/Documents/PyMOTW/src.pdb/PyMOTW/pdb/pdb_step.py(10)f()
-> for i in range(n):

 (The difference between next and step is that step stops inside a called function, while next executes called functions at (nearly) full speed, only stopping at the next line in the current function.)

## 3 Breakpoints

As programs grow even longer, even using `next` and `until` will become slow and cumbersome. 

Instead of stepping through the program by hand, a better solution is to let it run normally until it reaches a point where you want the debugger to interrupt it. 

You could use `set_trace()` to start the debugger, but that <b>only works if there is only one point you want to pause the program</b>. 

It is more convenient to run the program through the debugger, but tell the debugger where to stop in advance using <b>breakpoints</b>. 

The debugger monitors your program, and when it reaches the location described by a breakpoint the program is paused before the line is executed.

```
b(reak) [([filename:]lineno | function) [, condition]]
```
With a lineno argument, set a break there in the current file. With a function argument, set a break at the first executable statement within that function. The line number may be prefixed with a filename and a colon, to specify a breakpoint in another file (probably one that hasn’t been loaded yet). The file is searched on sys.path. Note that each breakpoint is assigned a number to which all the other breakpoint commands refer.

If a second argument is present, it is an expression which must evaluate to true before the breakpoint is honored.

Without argument, list all breaks, including for each breakpoint, the number of times that breakpoint has been hit, the current ignore count, and the associated condition if any.


<ol>
<li><b> c(ont(inue))</b> :Continue execution, only stop when a breakpoint is encountered.
<li><b> r(eturn)</b> :Continue execution until the current function returns.
</ol> 


In [4]:
def calc(i, n):
    j = i * n
    print('j =', j)
    if j > 0:
        print('Positive!')
    return j

def f(n):
    for i in range(n):
        print('i =', i)
        j = calc(i, n)
    return

if __name__ == '__main__':
    f(5)

i = 0
j = 0
i = 1
j = 5
Positive!
i = 2
j = 10
Positive!
i = 3
j = 15
Positive!
i = 4
j = 20
Positive!


There are several options to the break command used for setting break points. You can specify the line number, file, and function where processing should pause. To set a breakpoint on a specific line of the current file, use break lineno:

In [ ]:
$ python -m pdb pdb_break.py
> .../pdb_break.py(7)<module>()
-> def calc(i, n):
(Pdb) break 11
Breakpoint 1 at .../pdb_break.py:11

(Pdb) continue
i = 0
j = 0
i = 1
j = 5
> .../pdb_break.py(11)calc()
-> print 'Positive!'

(Pdb)

The command <b>continue</b> tells the debugger to keep running your program until the next breakpoint. 

In this case, it runs through the first iteration of the `for loop in f()` and stops inside `calc()` during the second iteration.

Breakpoints can also be set to the first line of a function by specifying the function name instead of a line number. This example shows what happens if a breakpoint is added for the `calc()` function.


In [ ]:
$ python -m pdb pdb_break.py
> .../pdb_break.py(7)<module>()
-> def calc(i, n):
(Pdb) break calc
Breakpoint 1 at .../pdb_break.py:7

(Pdb) continue
i = 0
> .../pdb_break.py(8)calc()
-> j = i * n

(Pdb) where
  .../pdb_break.py(21)<module>()
-> f(5)
  .../pdb_break.py(17)f()
-> j = calc(i, n)
> .../pdb_break.py(8)calc()
-> j = i * n

(Pdb)

## 6 Saving Configuration Settings

Debugging a program involves <b>a lot of repetition</b>; running the code, observing the output, adjusting the code or inputs, and running it again. 

<b>pdb</b> attempts to cut down on the amount of repetition you need to use to control the debugging experience, to let you concentrate on your code instead of the debugger. 

To help reduce the number of times you issue the same commands to the debugger, pdb lets you save configuration using text files read and interpreted on startup.

The file ~/.pdbrc is read first, allowing you to set global personal preferences for all of your debugging sessions. Then ./.pdbrc is read from the current working directory, so you can set local preferences for a particular project.


In [ ]:
$ cat ~/.pdbrc
# Show python help
alias ph !help(%1)
# Overridden alias
alias redefined p 'home definition'

$ cat .pdbrc
# Breakpoints
break 10
# Overridden alias
alias redefined p 'local definition'

$ python -m pdb pdb_function_arguments.py
Breakpoint 1 at .../pdb_function_arguments.py:10
> .../pdb_function_arguments.py(7)<module>()
-> import pdb
(Pdb) alias
ph = !help(%1)
redefined = p 'local definition'

(Pdb) break
Num Type         Disp Enb   Where
1   breakpoint   keep yes   at .../pdb_function_arguments.py:10


Any configuration commands that can be typed at the debugger prompt can be saved in one of the startup files,

If a file .pdbrc exists in the user’s home directory or in the current directory, it is read in and executed as if it had been typed at the debugger prompt. This is particularly useful for aliases. If both files exist, the one in the home directory is read first and aliases defined there can be overridden by the local file.


Changed in version 3.2: .pdbrc can now contain commands that continue debugging, such as continue or next
